# Import libraries

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import pickle

Using TensorFlow backend.


# Load processed dataset
The data we load here are already shuffled and normalized.

In [2]:
data_directory = 'dataset/8. oxford-normalized-30/'
x_train = pickle.load(open(data_directory + "x_train.p", "rb"))
y_train = pickle.load(open(data_directory + "y_train.p", "rb"))
x_test = pickle.load(open(data_directory + "x_test.p", "rb"))
y_test = pickle.load(open(data_directory + "y_test.p", "rb"))
print('x_train: ' + str(x_train.shape))
print('y_train: ' + str(y_train.shape))
print('x_test: ' + str(x_test.shape))
print('y_test: ' + str(y_test.shape))

x_train: (136, 22)
y_train: (136, 2)
x_test: (59, 22)
y_test: (59, 2)


# Tunable parameters

In [3]:
epochs = 10000
batch_size = 20
learning_rate = 0.0001
decay = 0.0001
loss = keras.losses.mean_squared_error

# Autoencoder #1

In [4]:
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
root_directory = 'results/59-oxford-autoencoder-1-30-01/'
model_directory = root_directory + "model/"
tensorboard_directory = root_directory + "tensorboard/"
def create_model(layer_size, dropout):
    global date
    x = Input(shape=(x_train.shape[1],))

    y = Dense(units = layer_size, activation='relu')(x)
    y = Dropout(dropout)(y)

    y = Dense(units = x_train.shape[1], activation='sigmoid')(y)
    model = Model(x, y)
    
    # Create directory
    directory = model_directory + date + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Write model hyper-parameters
    df = pd.DataFrame({
        'Hidden layer units': layer_size,
        'Dropout': dropout,
        'Learning Rate': learning_rate,
        'Decay': decay,
        'Epochs': epochs,
        'Batch Size': batch_size,
        'Loss': loss,
        'Test size': 30
    }, index=[0])
    df.to_csv(model_directory + date + '/params.csv')
    
    # Write model summary
    file2 = open(directory + "summary.txt", "a")
    model.summary(print_fn=lambda line: file2.write(line + '\n'))
    file2.close()

    # Write model diagram
    plot_model(model, to_file=directory + 'model.png', show_shapes=True, show_layer_names=False)
    
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate, decay=decay), loss=loss, metrics=['accuracy'])
    
    return model

# Prepare callbacks

In [5]:
class KerasClassifierTensorBoard(KerasClassifier):
    def fit(self, x, y, **kwargs):
        global date
        date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_directory + date + '/')
        csv_logger = keras.callbacks.CSVLogger(model_directory + date + '/epochs.csv')
        model_checkpoint = keras.callbacks.ModelCheckpoint(model_directory + date + '/weights.hdf5', save_best_only=True)
        callbacks = [tensorboard_callback, csv_logger, model_checkpoint]
        return super(KerasClassifierTensorBoard, self).fit(x, y, callbacks=callbacks, **kwargs)

# Grid search

In [6]:
model = KerasClassifierTensorBoard(build_fn=create_model, epochs=epochs, batch_size=batch_size)

layers = [10, 12, 8]
dropouts = [0.2, 0.4]
param_grid = dict(
    layer_size=layers, 
    dropout=dropouts
)

grid = GridSearchCV(estimator = model, param_grid = param_grid, error_score=0, verbose=2, cv=[(slice(None), slice(None))], n_jobs=1, fit_params=dict(validation_data=(x_test, x_test), verbose=0))
grid_result = grid.fit(x_train, x_train)

Fitting 1 folds for each of 6 candidates, totalling 6 fits
[CV] layer_size=10, dropout=0.2 ......................................
 20/136 [===>..........................] - ETA: 0s[CV] ....................... layer_size=10, dropout=0.2, total=14.7min
[CV] layer_size=12, dropout=0.2 ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 14.7min remaining:    0.0s


 20/136 [===>..........................] - ETA: 0s[CV] ....................... layer_size=12, dropout=0.2, total=15.7min
[CV] layer_size=8, dropout=0.2 .......................................
 20/136 [===>..........................] - ETA: 0s[CV] ........................ layer_size=8, dropout=0.2, total=15.9min
[CV] layer_size=10, dropout=0.4 ......................................
 20/136 [===>..........................] - ETA: 0s[CV] ....................... layer_size=10, dropout=0.4, total=14.5min
[CV] layer_size=12, dropout=0.4 ......................................
 20/136 [===>..........................] - ETA: 0s[CV] ....................... layer_size=12, dropout=0.4, total=13.2min
[CV] layer_size=8, dropout=0.4 .......................................
 20/136 [===>..........................] - ETA: 0s[CV] ........................ layer_size=8, dropout=0.4, total=13.7min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 87.7min finished
